In [1]:
from dotenv import load_dotenv
import os
import google.generativeai as genai
from langchain_community.document_loaders import PyPDFLoader

d:\LAB\lab1\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv() # Loads .env file
genai.configure(api_key=os.getenv("GOOGLE_API_KEY")) # Loads API key

In [3]:
loader = PyPDFLoader("ispt2.pdf")  # Load your PDF file
data = loader.load()
print(data)

[Document(metadata={'source': 'ispt2.pdf', 'page': 0}, page_content="L'Institut Supérieur Pédagogique et Technique de Kinshasa (ISPT -\nKIN) a été créé en 1976 dans le cadre de la coopération Belgo -Zaïroise. Il \nest destiné à former des ingénieurs doublés de qualifications Pédagogique \net Technique. Ils sont qualifiés  d'abord pour l'enseignement, mais ils sont \naussi qualifiés pour d'autres secteurs en dehors de l'enseignement.  \n \nIls sont dans des usines, des industries, dans l'administration du \nsecteur technique, etc.  Il s’agit de : \n \n\uf0b7 Les écoles techn iques ;  \n\uf0b7 Les écoles professionnelles ;  \n\uf0b7 Les centres de formation professionnelle ;  \n\uf0b7 Les unités de formation, de recyclage et de perfectionnement \ntechniques de diverses entreprises et sociétés du pays.  \n \nL'ISPT -KIN assure  également le rec yclage des enseignants \ntechniques du secondaire dans les disciplines qu'il assure maintenant à \nsavoir l'électrotechnique, l'informatique de ge

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)

docs = text_splitter.split_documents(data)

print("Total number of Chunks: ", len(docs))  # Check how many chunks we have
for chunk in docs:
    print(chunk.page_content)

Total number of Chunks:  6
L'Institut Supérieur Pédagogique et Technique de Kinshasa (ISPT -
KIN) a été créé en 1976 dans le cadre de la coopération Belgo -Zaïroise. Il 
est destiné à former des ingénieurs doublés de qualifications Pédagogique 
et Technique. Ils sont qualifiés  d'abord pour l'enseignement, mais ils sont 
aussi qualifiés pour d'autres secteurs en dehors de l'enseignement.  
 
Ils sont dans des usines, des industries, dans l'administration du 
secteur technique, etc.  Il s’agit de : 
 
 Les écoles techn iques ;  
 Les écoles professionnelles ;  
 Les centres de formation professionnelle ;  
 Les unités de formation, de recyclage et de perfectionnement 
techniques de diverses entreprises et sociétés du pays.  
 
L'ISPT -KIN assure  également le rec yclage des enseignants 
techniques du secondaire dans les disciplines qu'il assure maintenant à 
savoir l'électrotechnique, l'informatique de gestion, l'électronique, 
l'électromécanique, l'Informatique industrielle et rése

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from dotenv import load_dotenv
#from google.colab import userdata

# Load the Gemini API key
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Test embedding a query
vector = embeddings.embed_query("ISPT-KIN")
print(len(vector))
print(vector[0])

from langchain_chroma import Chroma

vectorstore1db = Chroma.from_documents(documents=docs, embedding=embeddings)
retrievers = vectorstore1db.as_retriever()

768
0.021842367947101593


In [6]:
retrieved_docs = retrievers.invoke( "Que signifie ISPT-KIN" ) 
print ( len (retrieved_docs)) 
print (retrieved_docs[ 0 ].page_content)   # Imprimer le premier document récupéré

4
- Electromécanique  
- Electronique  
- Télécommunication  
 
 En Section Informatique  :  
 
Nous avons  : 
 
- Informatique industrielle  
- Informatique de gestion  
- Réseaux et technique de  maintenance  
 
Il y a aussi un département Pyscho -pédagogie qui s ’occupe de l ’agrégation  
II.2.8. Les infrastructures de l’ISPT -KIN 
 
En ce qui concerne les infrastructures, nous signalons que l’ISPTKIN 
compte en ce jour sept bâtiments, à savoir :  
 
- Le bâtiment de la direction générale  nouvellement construite  
- Le bâtiment du home réh abilité  en auditoires  
- Les deux bâtiment s de l’EDAP   
- Le bâtiment de l’atelier mécanique  en construction  
- La grande salle  polyvalente ...  
 
Site web : https://www.isptkin.ac.cd/  
Contact  : info@isptkin.ac.cd  
L’ISPT-KIN a environ 18000 étudiants  
Le frais académique pour l’année 2023 -2024 a couté 300 dollars 
américains. Pour cette année, il faut contacter les services de l ’ISPT.  
 
Directeur Général  actuel  : DG P.O KABAS

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)


In [10]:
from langchain.chains import create_retrieval_chain 
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate 

# Définir une invite système
system_prompt = ( 
   "Vous êtes un Assistant spécialisé dans les renseignements pour l'établissement ISPT-KIN. Fournissez des réponses claires en fonction du contexte fourni. " 
    "Si les informations ne sont pas trouvées dans le contexte, indiquez que la réponse n'est pas disponible. " 
    
   
    "\n\n" 
    "{context}"
 ) 

# Configurer l'invite pour la chaîne d'assurance qualité
prompt = ChatPromptTemplate.from_messages( 
    [ 
        ( "system" , system_prompt), 
        ( "human" , "{input}" ) 
    ] 
) 

# Créer la chaîne RAG chain
# Create the RAG chain
chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retrievers, chain)

In [11]:
response = rag_chain.invoke({"input": "Que signifie ISPT-KIN"})
print(response)

{'input': 'Que signifie ISPT-KIN', 'context': [Document(metadata={'page': 5, 'source': 'ispt2.pdf'}, page_content='- Electromécanique  \n- Electronique  \n- Télécommunication  \n \n\uf0d8 En Section Informatique  :  \n \nNous avons  : \n \n- Informatique industrielle  \n- Informatique de gestion  \n- Réseaux et technique de  maintenance  \n \nIl y a aussi un département Pyscho -pédagogie qui s ’occupe de l ’agrégation  \nII.2.8. Les infrastructures de l’ISPT -KIN \n \nEn ce qui concerne les infrastructures, nous signalons que l’ISPTKIN \ncompte en ce jour sept bâtiments, à savoir :  \n \n- Le bâtiment de la direction générale  nouvellement construite  \n- Le bâtiment du home réh abilité  en auditoires  \n- Les deux bâtiment s de l’EDAP   \n- Le bâtiment de l’atelier mécanique  en construction  \n- La grande salle  polyvalente ...  \n \nSite web : https://www.isptkin.ac.cd/  \nContact  : info@isptkin.ac.cd  \nL’ISPT-KIN a environ 18000 étudiants  \nLe frais académique pour l’année 2023 